This is a guidance to generate a international competititon network from N national network.
* the N national network were created by CountryLeague Class, see dfg_rating/experiments/test_international_network/generate_national_competition.ipynb
* Then the international competition were scheduled between the teams choosen from all nations' 1st division teams by their pre-season's ranking (end-of-season points).
* The schedule of the international matches was also controled by the logic of min_match_per_team and avg_match_per_team

First step, import the needed module

In [ ]:
from dfg_rating.model.network.international_network import CountryLeague, InternationalCompetition_Combine
from dfg_rating.model.rating.multi_mode_rating import ControlledRandomFunction, ControlledTrendRating
from dfg_rating.model.forecast.true_forecast import LogFunctionForecast

Then, creating national networks as guideline: dfg_rating/experiments/test_international_network/generate_national_competition.ipynb
- Note the country id should not be same. Keep seasons same.

Here just create two national networks as example:

In [ ]:
rating_level1 = ControlledTrendRating(
    starting_point=ControlledRandomFunction(distribution='normal', loc=800, scale=100),
    delta=ControlledRandomFunction(distribution='normal', loc=0, scale=.5),
    trend=ControlledRandomFunction(distribution='normal', loc=0, scale=.2),
    season_delta=ControlledRandomFunction(distribution='normal', loc=0, scale=30),
    rating_name='true_rating'
)

rating_level2 = ControlledTrendRating(
    starting_point=ControlledRandomFunction(distribution='normal', loc=500, scale=50),
    delta=ControlledRandomFunction(distribution='normal', loc=0, scale=.5),
    trend=ControlledRandomFunction(distribution='normal', loc=0, scale=.2),
    season_delta=ControlledRandomFunction(distribution='normal', loc=0, scale=30),
    rating_name='true_rating'
)

rating_level3 = ControlledTrendRating(
    starting_point=ControlledRandomFunction(distribution='normal', loc=200, scale=10),
    delta=ControlledRandomFunction(distribution='normal', loc=0, scale=.5),
    trend=ControlledRandomFunction(distribution='normal', loc=0, scale=.2),
    season_delta=ControlledRandomFunction(distribution='normal', loc=0, scale=30),
    rating_name='true_rating'
)

forecast_test = LogFunctionForecast(
    outcomes=['home', 'draw', 'away'],
    coefficients=[-0.9, 0.3],
    beta_parameter=0.006
)

test_network1 = CountryLeague(
    teams=16,
    level1_teams=6,
    level2_teams=4,
    level3_teams=4,
    promotion_number=1,
    true_rating_level1=rating_level1,
    true_rating_level2=rating_level2,
    true_rating_level3=rating_level3,
    true_forecast=forecast_test,
    seasons=3,
    country_id='c1',
    rating_mode='interchange',
    min_match_per_team_level1_level2=1,
    avg_match_per_team_level1_level2=4,
    min_match_per_team_level2_level3=1,
    avg_match_per_team_level2_level3=2,
    min_match_per_team_level3_level1=1,
    avg_match_per_team_level3_level1=3,
    avg_match_per_team_level1_level1=0.4,
    min_match_per_team_level2_level2=1,
    avg_match_per_team_level2_level2=0.2,
    min_match_per_team_level3_level3=1,
    avg_match_per_team_level3_level3=0.3,
)


test_network2 = CountryLeague(
    teams=14,
    level1_teams=4,
    level2_teams=4,
    level3_teams=4,
    promotion_number=1,
    true_rating_level1=rating_level1,
    true_rating_level2=rating_level2,
    true_rating_level3=rating_level3,
    true_forecast=forecast_test,
    seasons=3,
    country_id='c2',
    rating_mode='interchange',
    min_match_per_team_level1_level2=1,
    avg_match_per_team_level1_level2=4,
    min_match_per_team_level2_level3=1,
    avg_match_per_team_level2_level3=2,
    min_match_per_team_level3_level1=1,
    avg_match_per_team_level3_level1=3,
)

Then call the InternationalCompetition_Combine class to shedule international competition from the national network we just created.
* teams_per_country: means how many teams you want choose to join the international competition from a nation's 1st division
* seasons, choose_mode, avg_match_per_team, min_match_per_team are same in the national network config
* create_country_network: True means you want the international class create the national network inside. Default False.


In [ ]:
test_network = InternationalCompetition_Combine(
    countries_configs={'c1':test_network1, 'c2':test_network2},
    teams_per_country=2,
    seasons=3,
    choose_mode='top',
    create_country_network=False,
    avg_match_per_team=3,
    min_match_per_team=1,
)

Example of add elo rating:
elo rating will sort all matches happened in the international network by match day, every new team initials 1000 points.

In [ ]:
from dfg_rating.model.rating.multi_mode_rating import ELORating
elo_rating1 = ELORating(
    rating_name='elo_rating',
    trained=True,
    rating_mode='keep',
    rating_mean=1000,
)
test_network.add_rating(elo_rating1, 'elo_rating')

Then you can export the network.
The international network will export all League, national, and international matches.

In [ ]:
test_network.export(printing_ratings=['true_rating','ranking','elo_rating'],file_name='test_InternationalLeague_network.csv')
test_network1.export(printing_ratings=['true_rating','ranking','elo_rating'],file_name='test_NationalLeague_network.csv')